In [33]:
# -*- coding: utf-8 -*-
"""
@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
from selenium.webdriver.common.action_chains import ActionChains
import os
import warnings
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

warnings.filterwarnings("ignore")


In [34]:
#driver=webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')
from selenium import webdriver 



chrome_options = Options()

chrome_options.add_argument("--disable-extensions")
# linux only
# chrome_options.add_argument("--headless")
#chrome_options.headless = True 

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe',options=chrome_options)

In [35]:
url='http://selectbiosciences.com/conferences.aspx'

driver.get(url)
time.sleep(5)

soup = bs(driver.page_source,'lxml')
cards=soup.find_all('td',{'itemtype':'http://schema.org/Event'})
len(cards)

trus=[]
for m in cards:

    tit=m.find('p',{'class':'ic_category'}).text
    web=m.find('a')['href']
    if web.startswith('..'):
        web=web.replace('..','')
    link='http://selectbiosciences.com'+web
    lo=' '.join(m.find('p',{'class':'ic_date'}).find('span',{'itemprop':'location'}).text.split())
    ti=' '.join(m.find('p',{'class':'ic_date'}).text.split()).replace(lo,'')
    trus.append([tit,link,lo,ti])

In [39]:
select=[]
for k in trus:
    tit=k[0]
    link=k[1]
    lo=k[2]
    da=k[3]
    driver.get(link)
    time.sleep(6)
    soupn=bs(driver.page_source,'lxml')
    title=soupn.find('h1',{'class':'innerConf'}).text
    fish=soupn.find('h2',{'class':'innerConf confInfoHeaderSection'}).get_text(separator='{').replace('{-{','-').replace('Date:{','')
    dat=fish.split('{')[0]
    loc=fish.split('{')[-1]
    for m in ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']:
        if m in dat:
            dat=dat.replace(m,'')
    dat=dat.replace('–','-').replace('|','').replace('to','-').replace('TO','-').strip().replace('Oc-ber','October').replace('OC-BER','OCTOBER')
    if '(' in dat:
        dat=dat.split('(')[0].strip()
    if dat=='':
        start_date=end_date=''
    else:
        mad=dat
        #('September 8, 2022 I The Ritz-Carlton, Beijing',
    if re.search('[A-Sa-z]{3,9}\s?\d{1,2}\s+\d{4}\s?[A-Za-z]+\s?[A-Za-z]+\s?[A-Za-z]+',mad.replace(',','')):
        #print('juice')
        tad=re.search('[A-Sa-z]{3,9}\s?\d{1,2}\s+\d{4}',mad.replace(',','')).group()
        mad=tad
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '-' in mad:
        st=mad.split('-')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('-')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif 'and'in mad:
        st=mad.split('and')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('and')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    elif '&' in mad:
        st=mad.split('&')[0].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.split('&')[1].strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

    else:
        #print(mad)
        st=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')
        en=mad.strip().replace(',','').replace('nd','').replace('st','').replace('th','').replace('rd','').replace('ugu','ugust')

            
    if any(c.isalpha() for c in st)==False:
    #      print('leg')
        pa=re.search('[A-Sa-z]+\W+(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en
    #                        02        April      2022
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}\s\d{4}',st):
     #   print('awujale')
        start=st
        end=en

    #*                        April       02      2022
    elif re.search('[A-Sa-z]{3,9}\s\d{1,2}\s+\d{4}',st):
    #       print('ala')
        pa=re.search('(\d{4})',st).group()
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()
        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba     

    #                      02     2022
    elif re.search('\d{1,2}\s+\d{4}',en):
    #      print('kum')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',st).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    #                      02         April
    elif re.search('\d{1,2}\s+[A-Sa-z]{3,9}',st):
    #        print('is')
        pa=re.search('(\d{4})',en)#, maxsplit=0)
        sapa=pa.group()
        start=st+' '+sapa
        end=en

    #*                      April           02
    elif re.search('[A-Sa-z]{3,9}\s+\d{1,2}',st):
    #      print('bad')
        pa=re.search('(\d{4})',en).group()#, maxsplit=0)
        sa=re.search('([A-Sa-z]{3,9})',st).group()
        ta=re.search('(\d{1,2})',st).group()

        start=ta+' '+sa+' '+pa
        #
        ba=re.search('(\d{4})',en).group()
        ca=re.search('([A-Sa-z]{3,9})',en).group()
        da=re.search('(\d{1,2})',en).group()
        end=da+' '+ca+' '+ba

    else:
    #     print('shik')
        start=end=''
    if start=='':
        start_date=end_date=''
    else:
        pick=[start, end]
        try:
            spl_dt_obj = [datetime.strptime(v.strip(), '%d %B %Y') for v in pick]
        except:
            spl_dt_obj = [datetime.strptime(v.strip(), '%d %b %Y') for v in pick]
        date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        start_date=date_[0]
        end_date=date_[1]
        
    sp=soupn.find('div',{'id':'topTierKeynoteSection'}).find_all('p',{'class':'innerConf'})
    spea=[]
    for z in sp:
        na=' '.join(z.find('a').text.split())
        titl=' '.join(z.find('span',{'itemprop':'jobTitle'}).text.split())
        ln='http://selectbiosciences.com'+(z.find('a')['href']).replace('..','')
        tri={
            'name':na,
            'title':titl,
            'link':ln
        }
        spea.append(tri)
    Speakerlist=json.dumps(spea,ensure_ascii=False)
    inft=' '.join(soupn.find('div',{'class':'innerConfWelcome'}).h2.text.split())
    info=' '.join(soupn.find('div',{'class':'innerConfWelcome'}).text.split()).replace(inft,'').strip()
    mail_=json.dumps([' '.join(soupn.find('div',{'class':'ConferenceMainBox'}).find('p',{'class':'innerConf'}).a.text.split())],ensure_ascii=False)
    spo=soupn.find('table',{'class':'sponserBox'}).find_all('div')
    spon=[]
    for n in spo:
        tis=n.find('img')['alt']
        spon.append(tis)
    sponsor=json.dumps(spon,ensure_ascii=False)
    
    def gu(luc):
        google_url_for_location="https://www.google.com/search?q="+luc+"&oq="+luc+"&num=1"
        time.sleep(randint(0,3))
        driver.get(google_url_for_location)
        time.sleep(4)
        try:
            google_map_url=driver.find_element_by_id('lu_map').click()
        except:
            try:
                google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
            except:
                google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
        time.sleep(1)
        google_map_url=driver.current_url
#                 print(google_map_url)
        return(google_map_url)
        ######################################
    def get_google_map_url(location):
        try:
            return(gu(location))
        except:
            sha=location.split(',')
            try:
                gu(sha[-3])
            except:
                try:
                    gu(sha[-2])
                except:
                    try:
                        gu(sha[-1])
                    except Exception as e:
                        print(location, "; url didn't go through")
                        print(e)
                        return("")


    def countr(locale):
        try:
            google_url_for_location="https://www.google.com/search?q="+locale+"&oq="+locale+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            time.sleep(4)
            try:
                google_map_url=driver.find_element_by_id('lu_map').click()
            except:
                try:
                    google_map_url=driver.find_element_by_class_name('Xm7sWb').click()
                except:
                    google_map_url=driver.find_element_by_class_name('Lx2b0d').click()
            time.sleep(1)
            try:
                country=driver.find_element_by_class_name('x3AX1-LfntMc-header-title-VdSJob').text
            except:
                country=driver.find_element_by_class_name('bwoZTb').text
#                 print(country)
            return(country)
        except Exception as e:
            print(locale, ";country didn't go through")
            print(e)
            return("")
    ticket_list=''
    org_name='Select Biosciences'
    org_web='http://selectbiosciences.com/'
    org_pro='Established in 1999, Select Biosciences provides focused market reports, custom consultancy, training courses and conferences to the biomedical marketplace.'
    logo=''    
    time_=''
    agendalist=''
    type_=''
    category=''
    event_info=info
    event_web=link
    
    if 'nline' in loc or 'ff Campus' in loc or loc=='' or loc==' ' or loc=='-' or loc=='Virtual' or 'nspecified' in loc or 'irtual' in loc:
        on_off='1'
        city=''
        venue=''
        country=''
        googlePlaceUrl=''
    else:
        on_off='0'
        if ',' in loc:
            city=loc.split(',')[-2]+', '+loc.split(',')[-1].strip()
            country=loc.split(',')[-1].strip()
        else:
            city=loc
            country=countr(city)
        venue=loc
        googlePlaceUrl=get_google_map_url(venue)
    
    
    
    select.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])


In [40]:
len(select)

19

In [41]:
select

[['http://selectbiosciences.com/conferences/index.aspx?conf=LOACEU2022',
  'Lab-on-a-Chip and Microfluidics Europe 2022',
  '2022-06-21',
  '2022-06-22',
  '',
  'SelectBIO Lab-on-a-Chip and Microfluidics Europe 2022 now in its 15th year brings together researchers and industry participants from both academia and industry focusing on technology and innovation in the Lab-on-a-Chip (LOAC) and Microfluidics fields.Presentations will explore the latest advances in the Lab-on-a-Chip and Microfluidics Fields. Focus at this conference will also be given to some of the many applications of Lab-on-a-Chip, from life science research, to taking diagnostics to the point-of-care/point-of-need and body-on-a-chip/organs-on-a-chip.We focus on LOAC device production technologies, novel designs/technologies for manufacture, as well as the key application areas for LOAC from research to diagnostics as well as 3D-bioprinting and the convergence of microfluidics technologies with biofabrication and 3D-prin

In [42]:
select_dict=select


select_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=select_dict)
select_df.to_csv("select.tsv", sep = '\t',index=False)
print('Dusted')

Dusted


In [31]:
# driver.switch_to.window(driver.window_handles[0])

In [15]:
# for k in range(1,20):
#     driver.switch_to.window(driver.window_handles[k])
#     time.sleep(2)
#     soupn=bs(driver.page_source,'lxml')

In [14]:
# s=1
# for k in select:
#     #time.sleep(1)
#     driver.execute_script("window.open('');")
#     driver.switch_to.window(driver.window_handles[s])
#     driver.get(k[1])
#     s+=1
# print(s)

20
